In [1]:
from sklearn.preprocessing import MultiLabelBinarizer
from ast import literal_eval
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
import gensim
import scipy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [2]:
train = pd.read_csv('products-shelves-tagging-dataset/train.tsv', sep='\t')
test = pd.read_csv('products-shelves-tagging-dataset/test.tsv', sep='\t')

In [3]:
train['Product Long Description'] = train['Product Long Description'].fillna('')
train['Product Short Description'] = train['Product Short Description'].fillna('')
train['Short Description'] = train['Short Description'].fillna('')
train['Recommended Location'] = train['Recommended Location'].fillna('')
train['Recommended Use'] = train['Recommended Use'].fillna('')

test['Product Long Description'] = test['Product Long Description'].fillna('')
test['Product Short Description'] = test['Product Short Description'].fillna('')
test['Short Description'] = test['Short Description'].fillna('')
test['Recommended Location'] = test['Recommended Location'].fillna('')
test['Recommended Use'] = test['Recommended Use'].fillna('')

In [4]:
for i,row in train.iterrows():
    if type(row['Product Name']) is not float:
        complete_info = row['Product Name'] + ' ' + row['Short Description'] + ' ' + row['Product Short Description'] + ' ' + row['Product Long Description'] + ' ' + row['Recommended Location'] + ' ' + row['Recommended Use']
        train.loc[i,'Product_Info'] = complete_info
        
for i,row in test.iterrows():
    if type(row['Product Name']) is not float:
        complete_info = row['Product Name'] + ' ' + row['Short Description'] + ' ' + row['Product Short Description'] + ' ' + row['Product Long Description'] + ' ' + row['Recommended Location'] + ' ' + row['Recommended Use']
        test.loc[i,'Product_Info'] = complete_info

In [5]:
train.drop(['actual_color','item_id','Seller','Color',
            'Product Long Description','Product Name','Product Short Description','Recommended Room',
            'Short Description'], axis = 1, inplace=True)
test.drop(['actual_color','Seller','Color',
            'Product Long Description','Product Name','Product Short Description','Recommended Room',
            'Short Description'], axis = 1, inplace=True)

In [6]:
train = train[~train.Product_Info.isnull()]

In [7]:
bad_input = []
for i,row in test.iterrows():
    if type(row.Product_Info) is float:
        bad_input.append(row.item_id)

In [8]:
itemid_test = test.item_id

In [9]:
test.drop(['item_id'], axis = 1, inplace=True)

In [10]:
test = test[~test.Product_Info.isnull()]

In [11]:
mlb = MultiLabelBinarizer()
train['tag'] = train.tag.apply(lambda x: [literal_eval(x)][0])
Y_train = mlb.fit_transform(train.tag)

In [12]:
for i,row in train.iterrows():
    train.loc[i,'Product_Class'] = 'Other'
    if type(row.Actors) is not float or type(row['MPAA Rating']) is not float or type(row['Aspect Ratio']) is not float:
        train.loc[i,'Product_Class'] = 'Movie'
    elif type(row['ISBN']) is float and type(row['Synopsis']) is not float:
        train.loc[i,'Product_Class'] = 'Movie'
    elif type(row['Actual Color']) is not float:
        train.loc[i,'Product_Class'] = 'Movie'
    elif type(row['Artist ID']) is not float or type(row['Genre ID']) is not float:
        train.loc[i,'Product_Class'] = 'Music'
    elif type(row['ISBN']) is float and type(row['Publisher']) is not float:
        train.loc[i,'Product_Class'] = 'VideoGame'
    if type(row['ISBN']) is not float and type(row['Synopsis']) is not float:
        train.loc[i,'Product_Class'] = 'Book'
    elif type(row['ISBN']) is not float or type(row['Literary Genre']) is not float:
        train.loc[i,'Product_Class'] = 'Book'
        
for i,row in test.iterrows():
    test.loc[i,'Product_Class'] = 'Other'
    if type(row.Actors) is not float or type(row['MPAA Rating']) is not float or type(row['Aspect Ratio']) is not float:
        test.loc[i,'Product_Class'] = 'Movie'
    elif type(row['ISBN']) is float and type(row['Synopsis']) is not float:
        test.loc[i,'Product_Class'] = 'Movie'
    elif type(row['Actual Color']) is not float:
        test.loc[i,'Product_Class'] = 'Movie'
    elif type(row['Artist ID']) is not float or type(row['Genre ID']) is not float:
        test.loc[i,'Product_Class'] = 'Music'
    elif type(row['ISBN']) is float and type(row['Publisher']) is not float:
        test.loc[i,'Product_Class'] = 'VideoGame'
    if type(row['ISBN']) is not float and type(row['Synopsis']) is not float:
        test.loc[i,'Product_Class'] = 'Book'
    elif type(row['ISBN']) is not float or type(row['Literary Genre']) is not float:
        test.loc[i,'Product_Class'] = 'Book'

In [13]:
for i, row in train.iterrows():
    if type(row.Actors) is not float:
        train.loc[i,'Actors'] = 1
    else:
        train.loc[i,'Actors'] = 0
    if type(row['MPAA Rating']) is not float:
        train.loc[i,'MPAA Rating'] = 1
    else:
        train.loc[i,'MPAA Rating'] = 0
    if type(row['Aspect Ratio']) is not float:
        train.loc[i,'Aspect Ratio'] = 1
    else:
        train.loc[i,'Aspect Ratio'] = 0
    if type(row.ISBN) is not float:
        train.loc[i,'ISBN'] = 1
    else:
        train.loc[i,'ISBN'] = 0
    if type(row.Synopsis) is not float:
        train.loc[i,'Synopsis'] = 1
    else:
        train.loc[i,'Synopsis'] = 0
    if type(row['Artist ID']) is not float:
        train.loc[i,'Artist ID'] = 1
    else:
        train.loc[i,'Artist ID'] = 0
    if type(row.Publisher) is not float:
        train.loc[i,'Publisher'] = 1
    else:
        train.loc[i,'Publisher'] = 0
    if type(row['Genre ID']) is not float:
        train.loc[i,'Genre ID'] = 1
    else:
        train.loc[i,'Genre ID'] = 0
    if type(row['Literary Genre']) is not float:
        train.loc[i,'Literary Genre'] = 1
    else:
        train.loc[i,'Literary Genre'] = 0
    if type(row['Actual Color']) is not float:
        train.loc[i,'Actual Color'] = 1
    else:
        train.loc[i,'Actual Color'] = 0
        
for i, row in test.iterrows():
    if type(row.Actors) is not float:
        test.loc[i,'Actors'] = 1
    else:
        test.loc[i,'Actors'] = 0
    if type(row['MPAA Rating']) is not float:
        test.loc[i,'MPAA Rating'] = 1
    else:
        test.loc[i,'MPAA Rating'] = 0
    if type(row['Aspect Ratio']) is not float:
        test.loc[i,'Aspect Ratio'] = 1
    else:
        test.loc[i,'Aspect Ratio'] = 0
    if type(row.ISBN) is not float:
        test.loc[i,'ISBN'] = 1
    else:
        test.loc[i,'ISBN'] = 0
    if type(row.Synopsis) is not float:
        test.loc[i,'Synopsis'] = 1
    else:
        test.loc[i,'Synopsis'] = 0
    if type(row['Artist ID']) is not float:
        test.loc[i,'Artist ID'] = 1
    else:
        test.loc[i,'Artist ID'] = 0
    if type(row.Publisher) is not float:
        test.loc[i,'Publisher'] = 1
    else:
        test.loc[i,'Publisher'] = 0
    if type(row['Genre ID']) is not float:
        test.loc[i,'Genre ID'] = 1
    else:
        test.loc[i,'Genre ID'] = 0
    if type(row['Literary Genre']) is not float:
        test.loc[i,'Literary Genre'] = 1
    else:
        test.loc[i,'Literary Genre'] = 0
    if type(row['Actual Color']) is not float:
        test.loc[i,'Actual Color'] = 1
    else:
        test.loc[i,'Actual Color'] = 0

In [14]:
train.drop(['tag'], axis = 1, inplace=True)

In [15]:
train.drop(['Recommended Use'], axis = 1, inplace=True)
test.drop(['Recommended Use'], axis = 1, inplace=True)

In [16]:
# train.drop(['Actors','MPAA Rating','Aspect Ratio','ISBN','Synopsis',
#            'Artist ID','Publisher','Genre ID','Literary Genre','tag'], axis = 1, inplace=True)
# test.drop(['Actors','MPAA Rating','Aspect Ratio','ISBN','Synopsis',
#            'Artist ID','Publisher','Genre ID','Literary Genre'], axis = 1, inplace=True)

In [17]:
for i, row in train.iterrows():
    if row.Product_Class in ['Movie','Book','VideoGame']:
        train.loc[i,'Recommended Location'] = 1
    else:
        train.loc[i,'Recommended Location'] = 0
for i, row in test.iterrows():
    if row.Product_Class in ['Movie','Book','VideoGame']:
        test.loc[i,'Recommended Location'] = 1
    else:
        test.loc[i,'Recommended Location'] = 0

In [18]:
for i, row in train.iterrows():
    soup = BeautifulSoup(row.Product_Info, 'html.parser')
    train.loc[i,'Product_Info'] = soup.get_text(separator=' ')

for i, row in test.iterrows():
    soup = BeautifulSoup(row.Product_Info, 'html.parser')
    test.loc[i,'Product_Info'] = soup.get_text(separator=' ')

C:\Users\hirpa\AppData\Local\Temp\ipykernel_19872\166664019.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row.Product_Info, 'html.parser')
C:\Users\hirpa\AppData\Local\Temp\ipykernel_19872\166664019.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row.Product_Info, 'html.parser')
C:\Users\hirpa\AppData\Local\Temp\ipykernel_19872\166664019.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row.Product_Info, 'html.parser')
C:\Users\hirpa\AppData\Local\Temp\ipykernel_19872\166664019.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to 

In [19]:
train['Product_Info'] = train['Product_Info'].str.lower()
train.Product_Info = train.Product_Info.str.replace(r'[0-9]',' ')
train.Product_Info = train.Product_Info.str.replace(r'https?.*?(\s|\n)',' ')
train.Product_Info = train.Product_Info.str.replace(r'[\w\.-]+@[\w\.-]+',' ')
train.Product_Info = train.Product_Info.str.replace(r'\.|\-|\+|\\|\/|\;|\:|\!|\_|\@|\#|\$|\%|\^|\&|\*|\(|\)|\{|\}|\[|\]|\||\<|\>|\"|\'|\,|\?',' ')
train.Product_Info = train.Product_Info.str.replace(r'\s[a-zA-Z]{2}\s|\s[a-zA-Z]{1}\s',' ')

test['Product_Info'] = test['Product_Info'].str.lower()
test.Product_Info = test.Product_Info.str.replace(r'[0-9]',' ')
test.Product_Info = test.Product_Info.str.replace(r'https?.*?(\s|\n)',' ')
test.Product_Info = test.Product_Info.str.replace(r'[\w\.-]+@[\w\.-]+',' ')
test.Product_Info = test.Product_Info.str.replace(r'\.|\-|\+|\\|\/|\;|\:|\!|\_|\@|\#|\$|\%|\^|\&|\*|\(|\)|\{|\}|\[|\]|\||\<|\>|\"|\'|\,|\?',' ')
test.Product_Info = test.Product_Info.str.replace(r'\s[a-zA-Z]{2}\s|\s[a-zA-Z]{1}\s',' ')

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hirpa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")

In [22]:
for i, row in train.iterrows():
    text = row.Product_Info.split()
    text = [word for word in text if word not in stopwords]
    # text = [stemmer.stem(t) for t in text]
    train.loc[i,'Product_Info'] = ' '.join(text)

for i, row in test.iterrows():
    text = row.Product_Info.split()
    text = [word for word in text if word not in stopwords]
    # text = [stemmer.stem(t) for t in text]
    test.loc[i,'Product_Info'] = ' '.join(text)

In [23]:
# train.Product_Info = train.Product_Info.str.replace(r'\s[a-zA-Z]{2}\s|\s[a-zA-Z]{1}\s',' ')
# test.Product_Info = test.Product_Info.str.replace(r'\s[a-zA-Z]{2}\s|\s[a-zA-Z]{1}\s',' ')

In [24]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.0, ngram_range= (1,2), max_features = 50000)

In [25]:
tfidf = tfidf_vectorizer.fit_transform(train.Product_Info)
tfidf_test = tfidf_vectorizer.transform(test.Product_Info)

In [26]:
dftr = pd.get_dummies(train[['Product_Class','Item Class ID']])
dfte = pd.get_dummies(test[['Product_Class','Item Class ID']])
dfte = dfte.reindex(columns = dftr.columns, fill_value=False)

In [27]:
train.drop(['Product_Class','Item Class ID','Product_Info'], axis = 1, inplace=True)
test.drop(['Product_Class','Item Class ID','Product_Info'], axis = 1, inplace=True)
# train.drop(['Product_Class','Item Class ID'], axis = 1, inplace=True)
# test.drop(['Product_Class','Item Class ID'], axis = 1, inplace=True)

In [28]:
train = train.apply(lambda x: pd.to_numeric(x))
test = test.apply(lambda x: pd.to_numeric(x))

In [29]:
print(dfte)

       Product_Class_Book  Product_Class_Movie  Product_Class_Other  \
0                   False                False                 True   
1                   False                False                 True   
2                   False                 True                False   
3                   False                 True                False   
4                   False                 True                False   
...                   ...                  ...                  ...   
10588               False                 True                False   
10589               False                False                 True   
10590               False                 True                False   
10591               False                False                 True   
10592               False                 True                False   

       Product_Class_VideoGame  Item Class ID_1  Item Class ID_11  \
0                        False             True             False   
1        

In [30]:
dftr = scipy.sparse.csr_matrix(dftr.values)
dfte = scipy.sparse.csr_matrix(dfte.values)
train = scipy.sparse.csr_matrix(train.values)
test = scipy.sparse.csr_matrix(test.values)

In [31]:
X = scipy.sparse.hstack([tfidf, dftr,train])
X_te = scipy.sparse.hstack([tfidf_test, dfte,test])

In [32]:
X

<10584x50039 sparse matrix of type '<class 'numpy.float64'>'
	with 1276521 stored elements in Compressed Sparse Row format>

In [33]:
classif = OneVsRestClassifier(SVC(kernel='linear'))
classif.fit(X, Y_train)

OneVsRestClassifier(estimator=SVC(kernel='linear'))

In [34]:
out = classif.predict(X_te)

In [35]:
mlbout = mlb.inverse_transform(out)

In [36]:
bad_input

[12928, 14691, 16725, 18767, 20014, 20802]

In [37]:
final = []
k = 0
for i in range(len(itemid_test)):  
    if itemid_test[i] in bad_input:
        final.append(tuple([itemid_test[i],list()]))
        k = k+1
    else:
        final.append(tuple([itemid_test[i],list(mlbout[i-k])]))

In [38]:
output = pd.DataFrame(final,columns=['item_id','tag'])

In [39]:
output.to_csv('products-shelves-tagging-dataset/tags.tsv',sep='\t',index=False)

In [40]:
# set(test['Item Class ID']).difference(set(train['Item Class ID']))

In [41]:
# train[train['Item Class ID'].isnull() & ~train['Product Name'].isnull()]

In [42]:
# for i,row in train[train['Item Class ID'].isnull() & ~train['Product Name'].isnull() & ~train['ISBN'].isnull()].iterrows():
#    train[train['Item Class ID'].isnull() & ~train['Product Name'].isnull() & ~train['ISBN'].isnull()].set_value(i,'Item Class ID',3)

In [43]:
# train[train['Item Class ID'].isnull() & ~train['Product Name'].isnull() & ~train['ISBN'].isnull()]

In [44]:
# train[~train['ISBN'].isnull()]['Item Class ID'].value_counts()